In [1]:
# <step1 크롤링: 네이버 플레이스 리뷰 크롤링>

# 네이버 플레이스 API가 종료됨에 따라, 감성 분류 데이터셋 확보 예제를 카카오맵 selenium 크롤링 예제로 변경
# 카카오맵의 가게 리뷰 데이터를 selenium과 chromedriver를 활용해 동적 크롤링합니다.

In [2]:
# <카카오맵 리뷰 크롤링> p.236~238

# -*- coding: utf-8 -*-

%matplotlib inline 
# 주피터 노트북을 생성한 브라우저에서 그림(rich output)을 볼 수있게 해줌.

import pandas as pd # pandas 라이브러리 import
import numpy as np # numpy 라이브러리 import
import matplotlib.pyplot as plt # matplotlib 라이브러리 import
import seaborn as sns # seaborn 라이브러리 import
import warnings
warnings.filterwarnings("ignore")

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
excutable_path = 'chromedriver.exe'

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다
driver = webdriver.Chrome(executable_path=excutable_path)

# 카카오 지도에 접속합니다
driver.get(source_url)

# 검색창에 검색어를 입력합니다
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
searchbox.send_keys("강남역 고기집")

# 검색버튼을 눌러서 결과를 가져옵니다
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# 검색 결과를 가져올 시간을 기다립니다
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다
html = driver.page_source

# BeautifulSoup을 이용하여 html 정보를 파싱합니다
soup = BeautifulSoup(html, "html.parser") # 가져온 html 소스를 BeautifulSoup 클래스의 soup객체로 변환
# 조건에 맞는 html 태그 모두 contents_div에 가져오기
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = [] # 빈 리스트 page_urls 생성
for moreview in moreviews: # 조건 필터링했던 태그 하나씩 반복
    page_url = moreview.get("href") # get()함수 사용해서 html의 속성 추출
    print(page_url) # page_url에 저장된 값 출력
    page_urls.append(page_url) # page_url 값 page_urls 리스트에 저장

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://place.map.kakao.com/85570955
https://place.map.kakao.com/1503746075
https://place.map.kakao.com/95713992
https://place.map.kakao.com/741391811
https://place.map.kakao.com/2011092566
https://place.map.kakao.com/2062959414
https://place.map.kakao.com/1648266796
https://place.map.kakao.com/13573220
https://place.map.kakao.com/168079537
https://place.map.kakao.com/263830255
https://place.map.kakao.com/27238067
https://place.map.kakao.com/26431943
https://place.map.kakao.com/1907052666
https://place.map.kakao.com/1052874675
https://place.map.kakao.com/1780387311
https://place.map.kakao.com/1576421052


In [11]:
# <가져온 맛집 리스트의 리뷰 정보 크롤링하기> p.239~240

columns = ['score', 'review'] # score와 review로 두개의 칼럼명 생성
df = pd.DataFrame(columns=columns) # 리스트를 데이터프레임 형태로 변환

driver = webdriver.Chrome(executable_path=excutable_path)  # 크롬 드라이버를 사용합니다
for page_url in page_urls: # 이전에 저장해뒀단 page_urls 리스트의 값들을 하나씩 반복
    
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    time.sleep(2)
    
    # 첫 페이지 리뷰를 크롤링합니다
    html = driver.page_source # 검색 결과의 페이지 소스를 가져옵니다
    soup = BeautifulSoup(html, 'html.parser') # 가져온 html 소스를 BeautifulSoup 클래스의 soup객체로 변환
    # 조건에 맞는 html 태그중 첫번째만 contents_div에 가져오기
    contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
    
    # 별점을 가져옵니다.
    # 조건에 맞는 모든 html 태그 rates에 가져오기
    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
    
    # 리뷰를 가져옵니다.
    # 조건에 맞는 모든 html 태그 reviews에 가져오기
    reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    # zip 함수 사용해서 순서대로 각각 rate와 review 짝지어줌. 튜플형태
    for rate, review in zip(rates, reviews):
        # rate는 텍스트 정보의 첫번째만 데이터값으로 가져오고, 
        # review는 조건에 맞는 첫번째 태그의 텍스트 정보만 데이터값으로 가져옴
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns) # 아까 만든 칼럼명(score, review)과 방금 정한 데이터값에 따라 시리즈값 생성
        df = df.append(series, ignore_index=True) # 시리즈 값들 추가해서 데이터 프레임 만들어줌
    
    # 2-5페이지의 리뷰를 크롤링합니다
    for button_num in range(2, 6):
        # 오류가 나지 않는 경우는, 1페이지 크롤링과 같은 방식으로 진행
        try:
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)
            
            # 페이지 리뷰를 크롤링합니다
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

            # 별점을 가져옵니다.
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

            # 리뷰를 가져옵니다.
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
        except:
            break   
# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

# 4점 이상의 리뷰는 긍정 리뷰, 3점 이하의 리뷰는 부정 리뷰로 평가합니다.
# lambda 함수 사용해서 score의 데이터값이 3보다 크면 y의 데이터값을 1로, 그렇지 않으면 0으로 저장.
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 3 else 0) 
print(df.shape) # shape 변수 사용하여 데이터프레임의 행.열 크기 튜플 형태로 반환
df.head() # 데이터 프레임의 상위 값 출력

(374, 3)


,score,review,y
0,1,"예약할 때는 룸을 주기로 하고 홀을 주고, 덥고, 직원들이 정신이 없어 그 가격에 ...",0
1,5,점심식사 잘했던곳.후식커피한잔 하기도 좋고 주차가능합니다. 음식 맛있고 직원분 친절...,1
2,5,新鮮でおいしいです。,1
3,4,녹는다 녹아,1
4,4,,1


In [12]:
# <step2 텍스트 전처리: 분류 모델 피처로 변환하기>

In [13]:
# <한글 텍스트로 전처리>

import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거
def text_cleaning(text):
    # 한글의 정규표현식으로 한글만 추출합니다.
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 부분 제거
    result = hangul.sub('', text) # sub 함수 사용해서 한글만 추출
    return result

# 함수를 적용하여 리뷰에서 한글만 추출합니다.
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x)) # 리뷰에서 한글만 추출 후 저장
del df['review'] # 기존 리뷰 컬럼은 데이터 프레임에서 삭제

# 한 글자 이상의 텍스트를 가지고 있는 데이터만 추출합니다
df = df[df['ko_text'].str.len() > 0]
df.head()

,score,y,ko_text
0,1,0,예약할 때는 룸을 주기로 하고 홀을 주고 덥고 직원들이 정신이 없어 그 가격에 내가...
1,5,1,점심식사 잘했던곳후식커피한잔 하기도 좋고 주차가능합니다 음식 맛있고 직원분 친절하여...
3,4,1,녹는다 녹아
5,5,1,흠잡을곳 없는 반찬이 정갈합니다
6,5,1,코로나 땜에 걱정했는데 방역수칙도 잘 지키시고 살치실이랑 등심 부드러워서 너무 좋아...


In [15]:
# <형태소 단위로 추출>

from konlpy.tag import Okt # konlpy.tag 라이브러리의 Okt 모듈 import

# konlpy라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt() # 인스턴스 생성
    pos = tagger.pos(x) # pos()함수 사용해서 형태소 분석
    pos = ['{}/{}'.format(word,tag) for word, tag in pos] # 형태소/품사 형식으로 리스트에 저장
    return pos

# 형태소 추출 동작을 테스트합니다.
result = get_pos(df['ko_text'][0]) # 리스트의 첫번째 줄만 뽑
print(result)

SystemError: java.lang.UnsatisfiedLinkError: Native Library C:\Users\User\miniconda3\Lib\site-packages\_jpype.cp39-win_amd64.pyd already loaded in another classloader